# Function for roofs

In [72]:
from math import *
from pyplasm import *

Funzione per eliminare gli errori dai punti 

In [73]:
def noError (number):
    if number%0.0001 == 0.0:
        return number
    tmp = number*10000
    if tmp%1 >= .9:
        tmp = ceil(tmp)
    else:
        tmp = floor(tmp)
    return tmp/10000


Funzione per verificare che il punto passato come parametro esiste nella lista

In [74]:
def pointExists (point, pointsList):
    i = 0
    for p in pointsList:
        if abs(p[0] - point[0]) < 0.0001 and abs(p[1] - point[1]) < 0.0001 and abs(p[2] - point[2]) < 0.0001:
            return i
        i += 1
    return -1

Funzione per il tetto. Aggiungo un parametro per l'offset (per gestirlo a seconda delle dimensioni del tetto)

In [75]:
def roof (points, cells, offset):
    #roof2D_0 = MKPOL([[(0,0,0),(10,0,0),(10,10,0),(0,10,0),(10,3,5),(10,7,5)], [[1,5,2],[2,5,6,3],[3,6,4],[4,6,5,1]],1])
    #piramide = MKPOL([[(0,0,0),(10,0,0),(10,10,0),(0,10,0),(5,5,5)],[[1,2,3,4,5]],1])
    fullRoof = MKPOL([points, cells, 1])
    #roofPieno = MKPOL([[[0,0,0],[2,0,0],[2,4,0],[0,4,0],[1,1,1],[1,3,1]],[[1,2,3,4,5,6]],1])
    #VIEW(piramide)
    sk2 = SKEL_2(fullRoof)
    uk = UKPOL(sk2)
    uk_0 = uk[0]

    listaPunti = []
    for val in uk[0]:
        if val[2] != 0:
            uk_0.append([val[0], val[1], 0])
    for val in uk_0:
        esiste = 0
        for l in listaPunti:
            if abs(val[0] - l[0]) < 0.0001 and abs(val[1] - l[1]) < 0.0001 and abs(val[2] - l[2]) < 0.0001 or val[0]%0.0001 != 0 and val[1]%0.0001 != 0 and val[2]%0.0001 != 0:
                esiste = 1
        if esiste != 1:
            listaPunti.append(val)
    #print listaPunti

    newPoints = []
    newCells = []
    for cell in uk[1]:
        tmp = []
        for index in cell:
            point = uk[0][index-1]
            point[0] = noError(point[0])
            point[1] = noError(point[1])
            point[2] = noError(point[2])
            i = pointExists(point, newPoints)
            if i > -1:
                tmp.append(i+1)
            else:
                newPoints.append(point)
                tmp.append(len(newPoints))
            if abs(point[2]) > 0.0001:
                newP = [point[0],point[1],0]
                j = pointExists(newP, newPoints)
                if j > -1:
                    tmp.append(j+1)
                else:
                    newPoints.append(newP)
                    tmp.append(len(newPoints))
        newCells.append(tmp)

    print newPoints
    print newCells
    b = MKPOL([newPoints, newCells, 1])
    
    skb = SKEL_1(b)
    roof = OFFSET(offset)(skb)
    #roof = OFFSET([.4,.4,.8])(skb)
    
    #skb = SKEL_2(b)
    #roof = skb
    return roof
    #DIM(a)
    #VIEW(a)

Esempio di un tetto semplice
<img src="tetto semplice.png">

In [76]:
points = [[0,0,0],[2,0,0],[2,4,0],[0,4,0],[1,1,1],[1,3,1]]
cells = [[1,2,3,4,5,6]]
offset = [.1,.1,.2]
#VIEW(roof(points, cells, offset))

Esempio di un tetto con due celle convesse
<img src="tetto doppio.png">

In [77]:
points = [[0,0,0],[2,0,0],[1,1,2],[0,6,0],[2,4,0],[6,4,0],[6,6,0],[1,5,2],[5,5,2]]
cells =[[1,2,3,4,5,8],[4,5,6,7,8,9]]
offset = [.1,.1,.2]
#VIEW(roof(points, cells, offset))

In [78]:
points = [[0,0,0],[2,0,0],[1,1,2],[0,6,0],[2,4,0],[6,4,0],[6,6,0],[1,5,2],[5,5,4],[1,5,4]]
cells =[[1,2,3,4,5,8],[4,5,6,7,10,9]]
offset = [.1,.1,.2]
VIEW(roof(points, cells, offset))

[[2.0, -0.0, 0.0], [1.0, 1.0, 2.0], [1.0, 1.0, 0], [0.0, -0.0, 0.0], [1.0, 5.0, 2.0], [1.0, 5.0, 0], [0.0, 6.0, 0.0], [2.0, 4.0, -0.0], [5.0, 5.0, 4.0], [5.0, 5.0, 0], [6.0, 6.0, -0.0], [6.0, 4.0, -0.0], [1.0, 5.0, 4.0]]
[[1, 2, 3, 4], [4, 2, 3, 5, 6, 7], [5, 6, 7, 8], [1, 2, 3, 5, 6, 8], [4, 1, 8, 7], [9, 10, 11, 12], [13, 6, 9, 10, 11, 7], [8, 13, 6, 7], [12, 9, 10, 13, 6, 8], [12, 8, 7, 11]]


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000BCA2330> >

In [79]:
h = 3
h2 = 4
points = [[0,0,0],[0,10,0],[6,0,0],
          [10,0,0],[16,0,0],[6,2,0],
          [10,2,0],[3,2,h],[3,4,h],
          [4,6,h2],[11,6,h2],[13,4,h],
          [13,2,h],[12,7,h2],[12,8,h2],
          [16,11,0],[8,11,0],[8,10,0],
          [0,2,0],[16,2,0],[8,2,0]]
cells = [[1,3,8],[1,19,9,8],[3,6,9,8],[19,2,10,9],[2,10,11,18],[]]
offset = [.1,.1,.2]
#cells = [[1,3,8,9,6,19],[4,5,20,12,13,7],[2,19,6,9,21,11,18,10]]
#VIEW (roof(points,cells, offset))

Esempio di un tetto composto da più elementi
<img src="https://conceptdraw.com/a333c4/p1/preview/640/pict--landscape-design-garden-hotel-site-plan.png--diagram-flowchart-example.png">
<img src="tetto composito.png">

In [80]:
h0 = 3
h1 = 3
h2 = 2*h1
points = [[4,0,0],[28,0,0],[28,16,0],[4,16,0],[12,8,h2],[20,8,h2],
          [0,4,0],[4,4,0],[0,12,0],[4,12,0],[4,8,h1],[8,8,h1],
          [11,16,0],[11,22,0],[21,22,0],[21,16,0],[16,12,h1],[16,18,h1]]
cells =[[1,2,3,4,5,6],[7,8,9,10,11,12],[13,14,15,16,17,18]]
p = [[28,4,0],[28,12,0],[32,12,0],[32,4,0],[28,8,h0]]
c = [[1,2,3,4,5]]
offset = [.1,.1,.2]
#VIEW(STRUCT([roof(p,c,offset), T(3)(6), roof(points,cells, offset)]))

Prova di un tetto ricavato da una base ottagonale

In [ ]:
l = 4
t = (l/2)*sqrt(2)
h = 6*l
pt = [[l,0,0], [t, -t,0],[0,-l,0],[-t, -t,0],[-l,0,0], [-t, t,0], [0,l,0], [t, t,0], [0,0,h]]
for p in pt:
    p[0] += l
    p[1] += l
    
cl = [[1,2,3,4,5,6,7,8,9]]
offset = [.1,.1,.2]
#VIEW(roof(pt,cl, offset))

Esempio di una struttura complessa, ispirato a Notre Dame
<img src="NDame skel1.png">

In [ ]:
l = 10
t = (l/2)*sqrt(2)
h = 3*l
offset = [.4,.4,.8]
htorre = 30
tower = [[l,0],[t, -t],[0,-l],[-t, -t],[-l,0],[-t, t],[0,l],[t, t]]
tetto = [[l,0,0],[t, -t,0],[0,-l,0],[-t, -t,0],[-l,0,0],[-t, t,0],[0,l,0],[t, t,0],[0,0,h]]
celle = [[1,2,3,4,5,6,7,8,9]]
for p in tower:
    p[0] += l
    p[1] += l
torre = MKPOL ([tower, [[1,2,3,4,5,6,7,8]],1])
torre = PROD ([torre,QUOTE([htorre])])
torre = OFFSET ([.8,.8,.8])(SKEL_1(torre))

base = OFFSET([.8,.8,.8])(SKEL_1(CUBOID ([20,20,70])))
torreCentrale = STRUCT([torre, T([1,2,3])([l,l,htorre]), roof(tetto, celle, offset)])
torreCentrale = STRUCT([base, T(3)(70), torreCentrale])

torreQuadrata = CUBOID([20,20,90])
torreQuadrata = OFFSET([.8,.8,.8])(SKEL_1(torreQuadrata))

#torri
l = 8
t = (l/2)*sqrt(2)
h = 4
tetto = [[l,0,0],[t, -t,0],[0,-l,0],[-t, -t,0],[-l,0,0],[-t, t,0],[0,l,0],[t, t,0],[0,0,h]]
celle = [[1,2,3,4,5,6,7,8,9]]
for p in tetto:
    p[0] += 10
    p[1] += 10
cupolaTorre = roof(tetto,celle, offset)
torre_lato = STRUCT([torreQuadrata, T(3)(90),cupolaTorre])
torre_lato = STRUCT([torre_lato, T(2)(40), torre_lato])

#edificio
ed = OFFSET([.8,.8,.8])(SKEL_1(CUBOID ([140,20,50])))
ed2 = OFFSET([.8,.8,.8])(SKEL_1(CUBOID ([20,80,50])))
edificio = STRUCT([T(2)(30),ed])
edificio = STRUCT([edificio, T(1)(70),ed2])

#tetto traslo lungo y di 10 inizialmente
ptTetto = [[20,20,0],[20,40,0],[20,30,20],[90,20,0],[90,30,20],[90,40,0],
           [100,40,20],[90,70,0],[100,70,20],[110,70,0],[110,40,0],
           [110,30,20],[160,40,0],[150,30,20],[160,20,0], [110,20,0],
           [100,20,20],[110,-10,0],[100,-10,20],[90,-10,0]]
clTetto = [[1,2,3,4,5,6],[6,7,8,9,10,11],[11,12,13,14,15,16],[16,17,18,19,20,4]]
for val in ptTetto:
    val[1] += 10
tettoEd = roof(ptTetto,clTetto, offset)

tot = STRUCT([edificio, T([1,2])([70,30]), torreCentrale])
tot = STRUCT([torre_lato, T([1,2])([20,-10]), tot])
tot = STRUCT([tot,T([2,3])([-10,50]),tettoEd])
#VIEW(tot)